In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
file_path1 = 'moweapi00080_2023_11_16\HDGeneratorIREG01.csv'
data1 = pd.read_csv(file_path1)

In [4]:
file_path2 = "moweapi00080_2023_11_16\WindgeberHD.csv"
wind_speed_data = pd.read_csv(file_path2)

In [5]:
data1.head()

,name,time,TURBINEID,DEVICETYPE,DEVICEID,IREG_UBUS,IREG_IBUS,IREG_SPEED,IREG_PWM,IREG_CHOPPER_PWM,IREG_IGBT_TEMP,IREG_MOTOR_TEMP
0,GeneratorIREG01,1698939803000000000,01,Generator,x001,0.5,-0.007,0,0,0,0,12
1,GeneratorIREG01,1698939813000000000,01,Generator,x001,0.3,-0.007,0,0,0,0,12
2,GeneratorIREG01,1698939823000000000,01,Generator,x001,0.2,-0.007,0,0,0,0,12
3,GeneratorIREG01,1698939833000000000,01,Generator,x001,0,-0.007,0,0,0,0,11
4,GeneratorIREG01,1698939843000000000,01,Generator,x001,0,-0.007,0,0,0,0,11


In [6]:
wind_speed_data.head()

,name,time,windspeed,windspeed_MW
0,windgeber,1698745499000000000,7.2,9
1,windgeber,1698745500000000000,7.2,9
2,windgeber,1698745501000000000,7.2,9
3,windgeber,1698745502000000000,6.8,8.9
4,windgeber,1698745503000000000,7.6,8.9


In [7]:
data1.drop(columns=['name'], inplace=True)
data1.drop(columns=['TURBINEID'], inplace=True)
data1.drop(columns=['DEVICETYPE'], inplace=True)
data1.drop(columns=['DEVICEID'], inplace=True)

In [8]:
wind_speed_data.drop(columns=['windspeed_MW'], inplace=True)
wind_speed_data.drop(columns=['name'], inplace=True)

In [9]:
wind_speed_data.head()

,time,windspeed
0,1698745499000000000,7.2
1,1698745500000000000,7.2
2,1698745501000000000,7.2
3,1698745502000000000,6.8
4,1698745503000000000,7.6


In [10]:
# data1.columns

In [11]:
data1.describe()

,time,IREG_UBUS,IREG_IBUS,IREG_SPEED,IREG_PWM,IREG_CHOPPER_PWM,IREG_IGBT_TEMP,IREG_MOTOR_TEMP
count,144006,144006,144006,144006,144006,144006,144006,144006
unique,143993,1427,1894,789,80,11,3,53
top,time,0,-0.007,0,0,0,0,10
freq,14,65161,97068,113941,72169,143978,143881,20780


In [12]:
print("data1 size: ",len(data1))
print("wind_speed_data size: ",len(wind_speed_data))

data1 size:  144006
wind_speed_data size:  1378140
data1 size:  144006
wind_speed_data size:  1378140


In [13]:
print(data1['time'].duplicated().sum())  # Number of duplicate entries in df1
print(wind_speed_data['time'].duplicated().sum())  # Number of duplicate entries in df2

13
136


In [14]:
data1 = data1.drop_duplicates(subset='time')
wind_speed_data = wind_speed_data.drop_duplicates(subset='time')

In [15]:
# Merging both DataFrames based on 'time' column, keeping only the intersection
merged_df = pd.merge(data1, wind_speed_data, on='time', how='inner')

In [16]:
print(len(data1['time'].unique()))
print(len(wind_speed_data['time'].unique()))
print(len(merged_df['time'].unique()))

143993
1378004
143706


In [17]:
merged_df.head()

,time,IREG_UBUS,IREG_IBUS,IREG_SPEED,IREG_PWM,IREG_CHOPPER_PWM,IREG_IGBT_TEMP,IREG_MOTOR_TEMP,windspeed
0,1698939803000000000,0.5,-0.007,0,0,0,0,12,5.2
1,1698939813000000000,0.3,-0.007,0,0,0,0,12,5.6
2,1698939823000000000,0.2,-0.007,0,0,0,0,12,5.2
3,1698939833000000000,0,-0.007,0,0,0,0,11,6
4,1698939843000000000,0,-0.007,0,0,0,0,11,6


In [18]:
len(merged_df)

143706

In [19]:
merged_df.describe()

,time,IREG_UBUS,IREG_IBUS,IREG_SPEED,IREG_PWM,IREG_CHOPPER_PWM,IREG_IGBT_TEMP,IREG_MOTOR_TEMP,windspeed
count,143706,143706,143706,143706,143706,143706,143706,143706,143706
unique,143706,1427,1894,789,80,11,3,53,48
top,1698939803000000000,0,-0.007,0,0,0,0,10,4
freq,1,65053,96895,113735,72046,143691,143594,20741,10061


## Filtering data for MPPT Mode
Because mapping is required only when the turbine is in MPPT mode

In [20]:
merged_df['IREG_SPEED'] = pd.to_numeric(merged_df['IREG_SPEED'], errors='coerce')

In [21]:
merged_df['IREG_SPEED']

0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
          ...  
143701    326.0
143702    361.0
143703    329.0
143704    379.0
143705    370.0
Name: IREG_SPEED, Length: 143706, dtype: float64

In [22]:
filtered_data = merged_df[(merged_df['IREG_SPEED'] >= 400) & (merged_df['IREG_SPEED'] <= 750)]

In [23]:
len(filtered_data)

22973

In [24]:
merged_df['IREG_SPEED'].max()

1348.0

In [25]:
filtered_data['IREG_SPEED'].max()

750.0

In [26]:
filtered_data['IREG_SPEED'].min()

400.0

In [27]:
filtered_data.reset_index(drop=True, inplace=True)

In [28]:
filtered_data

,time,IREG_UBUS,IREG_IBUS,IREG_SPEED,IREG_PWM,IREG_CHOPPER_PWM,IREG_IGBT_TEMP,IREG_MOTOR_TEMP,windspeed
0,1698940293000000000,201.1,0.99,696.0,62,0,0,12,5.2
1,1698940303000000000,198.3,1.485,701.0,64,0,0,12,4.4
2,1698940313000000000,204.8,1.321,733.0,61,0,0,13,6
3,1698940323000000000,198.2,0.46,621.0,63,0,0,14,4.8
4,1698940333000000000,198.6,0.247,588.0,64,0,0,14,5.2
...,...,...,...,...,...,...,...,...,...
22968,1700127112000000000,197.8,0.64,424.0,78,0,0,23,7.2
22969,1700127122000000000,198.8,0.541,466.0,75,0,0,23,7.6
22970,1700127132000000000,199.9,1.028,524.0,74,0,0,23,7.2
22971,1700127142000000000,201.9,0.59,485.0,73,0,0,24,7.2


In [29]:
data=filtered_data.drop(columns=['time', 'IREG_UBUS', 'IREG_IBUS'])

In [30]:
data

,IREG_SPEED,IREG_PWM,IREG_CHOPPER_PWM,IREG_IGBT_TEMP,IREG_MOTOR_TEMP,windspeed
0,696.0,62,0,0,12,5.2
1,701.0,64,0,0,12,4.4
2,733.0,61,0,0,13,6
3,621.0,63,0,0,14,4.8
4,588.0,64,0,0,14,5.2
...,...,...,...,...,...,...
22968,424.0,78,0,0,23,7.2
22969,466.0,75,0,0,23,7.6
22970,524.0,74,0,0,23,7.2
22971,485.0,73,0,0,24,7.2


In [31]:
data['IREG_PWM'].unique() # can be a factor for mapping

array(['62', '64', '61', '63', '68', '60', '66', '67', '65', '58', '59',
       '69', '73', '75', '80', '76', '74', '77', '72', '79', '78', '83',
       '82', '71', '70', '57', '81', '56', '54', '55', '85', '52', '84',
       '53', '87', '86', '0', '51', '28', '36'], dtype=object)

In [32]:
data['IREG_CHOPPER_PWM'].unique() # can be a factor for mapping

array(['0', '2', '3', '12', '10', '11', '1', '7', '8'], dtype=object)

In [33]:
data['IREG_IGBT_TEMP'].unique() # can be a factor for mapping

array(['0', '65535'], dtype=object)

In [34]:
data['IREG_MOTOR_TEMP'].unique() # can be a factor for mapping

array(['12', '13', '14', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '10', '11', '15', '8', '37', '38', '39', '40', '41',
       '42', '43', '44', '45', '46', '47', '48', '49', '50', '52', '51',
       '9', '7'], dtype=object)

## Normalizing the Data

In [35]:
from sklearn.preprocessing import MinMaxScaler
data_scaler = MinMaxScaler()
data_scaled = data_scaler.fit_transform(data)
data_scaled = pd.DataFrame(data_scaled, columns=data.columns)

In [36]:
data

,IREG_SPEED,IREG_PWM,IREG_CHOPPER_PWM,IREG_IGBT_TEMP,IREG_MOTOR_TEMP,windspeed
0,696.0,62,0,0,12,5.2
1,701.0,64,0,0,12,4.4
2,733.0,61,0,0,13,6
3,621.0,63,0,0,14,4.8
4,588.0,64,0,0,14,5.2
...,...,...,...,...,...,...
22968,424.0,78,0,0,23,7.2
22969,466.0,75,0,0,23,7.6
22970,524.0,74,0,0,23,7.2
22971,485.0,73,0,0,24,7.2


In [37]:
data_scaled

,IREG_SPEED,IREG_PWM,IREG_CHOPPER_PWM,IREG_IGBT_TEMP,IREG_MOTOR_TEMP,windspeed
0,0.845714,0.712644,0.0,0.0,0.111111,0.342105
1,0.860000,0.735632,0.0,0.0,0.111111,0.289474
2,0.951429,0.701149,0.0,0.0,0.133333,0.394737
3,0.631429,0.724138,0.0,0.0,0.155556,0.315789
4,0.537143,0.735632,0.0,0.0,0.155556,0.342105
...,...,...,...,...,...,...
22968,0.068571,0.896552,0.0,0.0,0.355556,0.473684
22969,0.188571,0.862069,0.0,0.0,0.355556,0.500000
22970,0.354286,0.850575,0.0,0.0,0.355556,0.473684
22971,0.242857,0.839080,0.0,0.0,0.377778,0.473684


## Splitting the data into label and features

In [38]:
y = data_scaled['IREG_SPEED']

In [39]:
y

0        0.845714
1        0.860000
2        0.951429
3        0.631429
4        0.537143
           ...   
22968    0.068571
22969    0.188571
22970    0.354286
22971    0.242857
22972    0.077143
Name: IREG_SPEED, Length: 22973, dtype: float64

In [40]:
X = data_scaled.drop(columns=['IREG_SPEED'])

In [41]:
X

,IREG_PWM,IREG_CHOPPER_PWM,IREG_IGBT_TEMP,IREG_MOTOR_TEMP,windspeed
0,0.712644,0.0,0.0,0.111111,0.342105
1,0.735632,0.0,0.0,0.111111,0.289474
2,0.701149,0.0,0.0,0.133333,0.394737
3,0.724138,0.0,0.0,0.155556,0.315789
4,0.735632,0.0,0.0,0.155556,0.342105
...,...,...,...,...,...
22968,0.896552,0.0,0.0,0.355556,0.473684
22969,0.862069,0.0,0.0,0.355556,0.500000
22970,0.850575,0.0,0.0,0.355556,0.473684
22971,0.839080,0.0,0.0,0.377778,0.473684


In [42]:
X['IREG_PWM'] = pd.to_numeric(X['IREG_PWM'], errors='coerce')
X['IREG_CHOPPER_PWM'] = pd.to_numeric(X['IREG_CHOPPER_PWM'], errors='coerce')
X['IREG_IGBT_TEMP'] = pd.to_numeric(X['IREG_IGBT_TEMP'], errors='coerce')
X['IREG_MOTOR_TEMP'] = pd.to_numeric(X['IREG_MOTOR_TEMP'], errors='coerce')
X['windspeed'] = pd.to_numeric(X['windspeed'], errors='coerce')

In [43]:
# X.to_csv('X.csv', index=False)

In [44]:
# y.to_csv('y.csv', index=False)

## Feature Engineering

In [45]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

## Train-Test Split

In [46]:
from sklearn.model_selection import train_test_split

In [47]:
train_X, val_X, train_y, val_y = train_test_split(X_poly, y, test_size=0.1, random_state=42)

## Using Random Forest Regressor

In [48]:
# from sklearn.ensemble import RandomForestRegressor

In [49]:
# model1=RandomForestRegressor(n_estimators=100, random_state=42)
# model1.fit(train_X, train_y)

In [50]:
# preds1 = model1.predict(val_X)

In [58]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [52]:
# print("MAE: ", mean_absolute_error(val_y, preds1))
# print("RMSE: ",np.sqrt(mean_squared_error(val_y, preds1)))
# print("R2 Score: ", r2_score(val_y, preds1))

## Using Neural Networks

In [60]:
from sklearn.neural_network import MLPRegressor

In [61]:
NN_model = MLPRegressor(hidden_layer_sizes=(64, 128, 64), max_iter=500, learning_rate='invscaling', verbose=True, )

In [62]:
NN_model.fit(train_X, train_y.values.ravel())

Iteration 1, loss = 0.02806789
Iteration 2, loss = 0.01319878
Iteration 3, loss = 0.01259865
Iteration 4, loss = 0.01218340
Iteration 5, loss = 0.01183252
Iteration 6, loss = 0.01176074
Iteration 7, loss = 0.01127224
Iteration 8, loss = 0.01108872
Iteration 9, loss = 0.01072283
Iteration 10, loss = 0.01016063
Iteration 11, loss = 0.00992148
Iteration 12, loss = 0.00973458
Iteration 13, loss = 0.00958167
Iteration 14, loss = 0.00927270
Iteration 15, loss = 0.00920039
Iteration 16, loss = 0.00905753
Iteration 17, loss = 0.00916207
Iteration 18, loss = 0.00911938
Iteration 19, loss = 0.00892605
Iteration 20, loss = 0.00877388
Iteration 21, loss = 0.00891826
Iteration 22, loss = 0.00876591
Iteration 23, loss = 0.00864640
Iteration 24, loss = 0.00854411
Iteration 25, loss = 0.00855948
Iteration 26, loss = 0.00850013
Iteration 27, loss = 0.00868289
Iteration 28, loss = 0.00835746
Iteration 29, loss = 0.00844836
Iteration 30, loss = 0.00846869
Iteration 31, loss = 0.00843595
Iteration 32, los

MLPRegressor(hidden_layer_sizes=(64, 128, 64), learning_rate='invscaling',
             max_iter=500, verbose=True)

In [63]:
preds6 = NN_model.predict(val_X)

In [64]:
print("MAE: ", mean_absolute_error(val_y, preds6))
print("RMSE: ",np.sqrt(mean_squared_error(val_y, preds6)))
print("R2 Score: ", r2_score(val_y, preds6))

MAE:  0.08945106236839087
RMSE:  0.1355997787411176
R2 Score:  0.6844108771427404
